<a href="https://colab.research.google.com/github/Ann-15/AI_Health_Assistant_ETL/blob/main/AI_Health_Assistant_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas sentence-transformers faiss-cpu

In [ ]:
import pandas as pd

data = {
    "Question": [
        "How to check blood pressure?",
        "How much water should I drink daily?",
        "How often should I exercise?"
    ],
    "Answer": [
        "Use a digital blood pressure monitor every morning.",
        "Drink at least 8 glasses of water every day.",
        "Exercise for 30 minutes, 5 days a week."
    ]
}

df = pd.DataFrame(data)



In [ ]:
from sentence_transformers import SentenceTransformer


model = SentenceTransformer('all-MiniLM-L6-v2')


documents = df['Question'] + " " + df['Answer']

embeddings = model.encode(documents.tolist())
embeddings

In [ ]:
import faiss
import numpy as np


dimension = embeddings.shape[1]


index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))



In [ ]:

query = "How can I monitor my blood pressure?"

query_vec = model.encode([query])


D, I = index.search(np.array(query_vec), k=1)

print("Question:", df['Question'][I[0][0]])
print("Answer:", df['Answer'][I[0][0]])